In [17]:
from mxnet import gluon
from mxnet.gluon import nn
from mxnet import autograd
from mxnet import nd
import numpy as np
import mxnet as mx
import matplotlib.pyplot as plt
from time import time
from mxnet import image
ctx = mx.gpu()

In [18]:
cifar10_train = gluon.data.vision.CIFAR10(root='/home/sinyer/python/data', train=True)
cifar10_test = gluon.data.vision.CIFAR10(root='/home/sinyer/python/data', train=False)

In [20]:
print(cifar10_test.shape)

AttributeError: 'CIFAR10' object has no attribute 'shape'

In [15]:
def get_data(batch_size):
    cifar10_train = gluon.data.vision.CIFAR10(root='/home/sinyer/python/data', train=True, transform=transform_train)
    cifar10_test = gluon.data.vision.CIFAR10(root='/home/sinyer/python/data', train=False, transform=transform_test)
    train_data = gluon.data.DataLoader(cifar10_train, batch_size, shuffle=True)
    test_data = gluon.data.DataLoader(cifar10_test, batch_size, shuffle=False)
    return train_data, test_data

train_data, test_data = get_data(50000)


In [12]:
def accuracy(output, label):
    return nd.mean(output.argmax(axis=1)==label).asscalar()

def evaluate_accuracy(data_iterator, net, ctx):
    acc = 0.
    for data, label in data_iterator:
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        output = net(data)
        acc += accuracy(output, label)
    return acc / len(data_iterator)

def relu(x):
    return nd.maximum(x, 0)
def net(x):
    x = x.reshape((-1, inp))
    h1 = relu(nd.dot(x, w1) + b1)
    output = nd.dot(h1, w2) + b2
    return output

In [13]:
n, inp, h, out = 50000, 3072, 128, 10
weight_scale = .01

w1 = nd.random_normal(shape=(inp, h), scale=weight_scale, ctx=ctx)
b1 = nd.zeros(h, ctx=ctx)
w2 = nd.random_normal(shape=(h, out), scale=weight_scale, ctx=ctx)
b2 = nd.zeros(out, ctx=ctx)

params = [w1, b1, w2, b2]
for param in params:
    param.attach_grad()

l = gluon.loss.SoftmaxCrossEntropyLoss()

In [14]:
lr = 0.01 / batch_size
epochs = 50

a = []
b = []
for epoch in range(epochs):
    if epoch > 30:
        lr = 0.001 / batch_size
    train_loss = 0.
    train_acc = 0.
    batch = 0
    start = time()
    for data, label in train_data:
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        with autograd.record():
            output = net(data)
            loss = l(output, label)
        loss.backward()
        for param in params:
            param[:] = param - lr * param.grad
        train_loss += nd.mean(loss).asscalar()
        train_acc += accuracy(output, label)
        batch += 1
    a.append(train_acc/batch)
    test_acc = evaluate_accuracy(test_data, net, ctx)
    b.append(test_acc)
    if epoch%2 == 0:
        print("E %d; L %f; Tr_acc %f; Te_acc %f; T %f" % (epoch, train_loss / batch, train_acc / batch, test_acc, time() - start))
plt.plot(np.arange(0, epochs),a,np.arange(0, epochs),b)
plt.ylim(0,1)
plt.show()


E 0; L 2.279837; Tr_acc 0.116665; Te_acc 0.185449; T 10.875642
E 2; L 2.070095; Tr_acc 0.262062; Te_acc 0.279687; T 11.001094
E 4; L 1.958817; Tr_acc 0.301606; Te_acc 0.294141; T 11.007186
E 6; L 1.903051; Tr_acc 0.328205; Te_acc 0.326562; T 11.919098
E 8; L 1.863060; Tr_acc 0.343575; Te_acc 0.339453; T 10.854677
E 10; L 1.828807; Tr_acc 0.356441; Te_acc 0.353906; T 10.965157
E 12; L 1.798464; Tr_acc 0.365860; Te_acc 0.361816; T 11.676691
E 14; L 1.771937; Tr_acc 0.376961; Te_acc 0.375488; T 11.792554
E 16; L 1.746129; Tr_acc 0.386324; Te_acc 0.382324; T 12.114321
E 18; L 1.721437; Tr_acc 0.396680; Te_acc 0.394043; T 11.561764
E 20; L 1.699706; Tr_acc 0.403691; Te_acc 0.403223; T 11.409371
E 22; L 1.678209; Tr_acc 0.411970; Te_acc 0.408789; T 11.129707
E 24; L 1.658459; Tr_acc 0.419507; Te_acc 0.412012; T 10.821159
E 26; L 1.639102; Tr_acc 0.427555; Te_acc 0.426270; T 12.109884
E 28; L 1.622228; Tr_acc 0.434829; Te_acc 0.430566; T 12.023988


KeyboardInterrupt: 